In [1]:
# environment setup

import os
import sys

os.environ["__KAGGLE__"] = "1"
sys.path.append("/kaggle/input/bc21-code/")

In [2]:
import argparse
import datetime
import os
import re
from collections import defaultdict
from glob import glob
from pprint import pformat

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tqdm import tqdm

from lib.utils import fix_random_seed, read_json
from src.config import c
from src.data_utils import (
    normalize_soundscapes_df,
    predictions_to_text_labels,
    read_soundscapes_info,
)
from src.generator import Generator
from src.models import Div, SinCos, YMToDate
from src.services import get_msg_provider, get_wave_provider

In [3]:
IN_KAGGLE_SUBMIT = os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Batch"

IN_CSV = (
    "/kaggle/input/birdclef-2021/test.csv"
    if IN_KAGGLE_SUBMIT
    else "/kaggle/input/birdclef-2021/fake_test.csv"
)

MODEL = "/kaggle/input/bc21-models/B1_nrsw_2.h5"

In [4]:
meta = read_json(MODEL.replace(".h5", ".json"))

In [7]:
df = pd.read_csv(IN_CSV)
df = normalize_soundscapes_df(df, quiet=True, seconds=5)

,filename,_primary_labels,_secondary_labels,_from_s,_to_s,_year,_month,latitude,longitude,rating,_source,row_id
0,7019_COR_20190904.ogg,,,0,5,2019,9,10.12,-84.51,5.0,long,7019_COR_5
1,7019_COR_20190904.ogg,,,5,10,2019,9,10.12,-84.51,5.0,long,7019_COR_10
2,7019_COR_20190904.ogg,,,10,15,2019,9,10.12,-84.51,5.0,long,7019_COR_15
3,7019_COR_20190904.ogg,,,15,20,2019,9,10.12,-84.51,5.0,long,7019_COR_20
4,7019_COR_20190904.ogg,,,20,25,2019,9,10.12,-84.51,5.0,long,7019_COR_25
...,...,...,...,...,...,...,...,...,...,...,...,...
2395,54955_SSW_20170617.ogg,,,575,580,2017,6,42.47,-76.45,5.0,long,54955_SSW_580
2396,54955_SSW_20170617.ogg,,,580,585,2017,6,42.47,-76.45,5.0,long,54955_SSW_585
2397,54955_SSW_20170617.ogg,,,585,590,2017,6,42.47,-76.45,5.0,long,54955_SSW_590
2398,54955_SSW_20170617.ogg,,,590,595,2017,6,42.47,-76.45,5.0,long,54955_SSW_595
